<h2>Padding sequence data</h2>
<h4>What is masking and padding?</h4>
<p><b>masking</b>: it is a way to tell sequence-processing layers that certain timesteps in an input are missing and those should be skipped when processing the data.<br>
<b>Padding</b>: it is a another form of masking where the masked steps are at the start or end of the sequence. In order to make all sequences in a batch to a given standard size, we need padding or truncating some sequence.</b></p>

In [4]:
#Importing required libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

<p>When we process sequence data, It is common thing to have samples with different lengths. Consider the example below,  where text is tokenized as words. </p>



```
[
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]
```




<p>After vocabulary lookup, the data might be vectorized as integers. For example,</p>



```
[
  [71, 1331, 4231]
  [73, 8, 3215, 55, 927],
  [83, 91, 1, 645, 1253, 927],
]
```



<p>In the above data, which is a nested list, where individual samples having different length like 3, 5 and 6. The input for a deep learning model must be a single tensor. The samples that are shorter than the longest item need to be padded with some placeholder value (example 0). Keras provides a utility function to truncate and pad python list to a common length. i.e. <b>tf.keras.preprocessing.sequence.pad_sequences</b>.</p>

In [5]:
# Sample data with different length of list
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# By default, this will pad using 0s; it is configurable via the
# value parameter.
# Note that you could pre padding (at the beginning) or
# post padding (at the end).
# We recommend using post padding when working with RNN layers
# (in order to be able to use the
# CuDNN implementation of the layers).
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)

# Displaying the results after padding
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


<h2>Masking</h2>
<p>Once the samples have a uniform length, the model must be informed that some data is padded with placeholder value and they must be ignored. This process is called masking. There are three ways to introduce masks in keras model:</p>
<ul><li>Add a keras.layers.Masking layer</li>
<li>Configure a keras.layers.Embedding layer with mask_zero = True</li>
<li>Pass mask argument manually when calling layers that support mask argument (example, RNN layers supports mask argument).</li></ul>

<h2>Mask-generating layers: Embedding and Masking</h2>
<p>Under the hood these layers will create a mask tensor (2D tensor with shape (batch, sequence_length)), and attach it to the tensor output returned by the Masking or Embedding layer.</p>

In [6]:
#Creating a embedding layer with mask_zero = True
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)

#Applying embedding to padded_inputs (Computed in previous section)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

#Creating masking layer
masking_layer = layers.Masking()

# Simulate the embedding lookup by expanding the 2D input to 3D,
# with embedding dimension of 10.
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


<p>From the above result, the mask is a 2D boolean tensor with shape (batch_size, sequence_length). The each False in output indicates that the corresponding timestep should be ignored during processing.</p> 